In [2]:
# if running for first time, you may need to install spacy model
# > python -m spacy download en_core_web_sm
import dotenv, pyreadr, os
dotenv.load_dotenv()

from pathlib import Path
from datetime import datetime
import pandas as pd
import sqlalchemy as sa
import sqlalchemy.orm as so

from hemonc_alchemy.matchers.spacy_config import matchers, nlp, rules, match_entities, get_modifier_child, get_nouns  
from hemonc_alchemy.model.hemonc_model import Hemonc_Study, Hemonc_Condition, Hemonc_Component, Hemonc_Modality, Hemonc_Component_Class, Hemonc_Regimen, Hemonc_Variant, Hemonc_Regimen_Part, Part_Phase, Hemonc_Cycle_Sig, Hemonc_Sig, Sig_Days, Base, component_to_class_map, Hemonc_Ref, Hemonc_Component_Role, Hemonc_Context, Hemonc_Branch_Conditional
from hemonc_alchemy.model.hemonc_enums import Intent, Setting, Phase, Risk, Phenotype, PriorTherapy, Phase, StudyDesign, SponsorType, BranchConditionalType, ComponentRole

oa_configurator.config - DEBUG - Application config path set: /Users/georginakennedy/cloudstor/CBDRH/ACDN/HemOnc_Alchemy/oa_system_config.yaml
oa_configurator.config - DEBUG - Log path set: /Users/georginakennedy/cloudstor/CBDRH/ACDN/HemOnc_Alchemy/logs
oa_configurator.config - DEBUG - DB connection string: sqlite:////Users/georginakennedy/cloudstor/CBDRH/ACDN/HemOnc_Alchemy/data/dash.db


In [3]:
# load data objects

# download the updated hemonc tables folder from dropbox and unzip it here
TABLE_PATH = Path('..') / 'Tables'

# download the required athena vocab files and place them here
CDM_PATH = Path('..') / 'OHDSI_Vocabs' 

In [4]:
concept_columns = ['concept_name', 'domain_id', 'vocabulary_id', 'concept_class_id', 'standard_concept', 'concept_code','valid_start_date', 'valid_end_date', 'invalid_reason']
concept_relationship_columns = ['concept_code_1', 'concept_code_2', 'vocabulary_id_1', 'vocabulary_id_2', 'relationship_id', 'valid_start_date', 'valid_end_date', 'invalid_reason']

sigs = pd.read_csv(TABLE_PATH / 'sigs.csv', low_memory=False)
omop = pyreadr.read_r(TABLE_PATH / 'omop.RData')
concept_stage = omop['concept_stage'][concept_columns].copy()
concept_relationship_stage = omop['concept_relationship_stage'][concept_relationship_columns].copy()
concept_synonym_stage = omop['concept_synonym_stage'].copy()

# using RData files where possible, to avoid versioning issues
context = pyreadr.read_r(TABLE_PATH / 'context.table.RData')['context.table']
page = pyreadr.read_r(TABLE_PATH / 'page.table.RData')['page.table']
pointer = pyreadr.read_r(TABLE_PATH / 'pointer.table.RData')['pointer.table']
ref = pyreadr.read_r(TABLE_PATH / 'ref.table.RData')['ref.table']
sequence = pyreadr.read_r(TABLE_PATH / 'sequence.table.RData')['sequence.table']
study = pyreadr.read_r(TABLE_PATH / 'study.table.RData')['study.table']

In [14]:
concept_synonym_stage[concept_synonym_stage.synonym_name.str.contains('\"')][50:]

,synonym_concept_id,synonym_name,synonym_concept_code,synonym_vocabulary_id,language_concept_id,valid_start_date,valid_end_date,invalid_reason
rownames,,,,,,,,
13812,,"20%"">Study",16023,HemOnc,4180186.0,2023-03-24,2099-12-31,
13816,,"20%"">Comparator",16023,HemOnc,4180186.0,2023-03-24,2099-12-31,
13817,,"100%; text-align:center;"">",16023,HemOnc,4180186.0,2023-03-24,2099-12-31,
13818,,"20%"">Dates of enrollment",16023,HemOnc,4180186.0,2023-04-12,2099-12-31,
13825,,"60%; text-align:center;"">",16028,HemOnc,4180186.0,2023-03-24,2099-12-31,
13827,,"33%"">Study",16028,HemOnc,4180186.0,2023-03-24,2099-12-31,
13828,,"33%"">Years of enrollment",16028,HemOnc,4180186.0,2023-03-24,2099-12-31,
13830,,"33%"">Dates of enrollment",16028,HemOnc,4180186.0,2023-04-12,2099-12-31,
13877,,"Rituximab, ""Mega"" (high-dose) Cyclophosphamide...",16106,HemOnc,4180186.0,2019-08-27,2099-12-31,


In [4]:
# author = pyreadr.read_r(TABLE_PATH / 'author.table.RData')['author.table']
# person = pyreadr.read_r(TABLE_PATH / 'person.table.RData')['person.table']
# variant = pyreadr.read_r(TABLE_PATH / 'variant.table.RData')['variant.table']

In [5]:
# following files had non-utf characters, requiring manual R-based conversion to csv because this is not handled with pyreadr at this time
author = pd.read_csv(TABLE_PATH / 'author.table.csv', low_memory=False)
person = pd.read_csv(TABLE_PATH / 'person.table.csv', encoding="ISO-8859-1")
variant = pd.read_csv(TABLE_PATH / 'variant.table.csv', encoding="ISO-8859-1")

regimen = pointer[pointer.regimen_cui!='NOT YET ASSIGNED'][['regimen', 'regimen_cui']].drop_duplicates().copy()

In [6]:
# load minimal athena download files for cross-reference including concept IDs where available
concepts_with_ids = pd.read_csv(CDM_PATH / 'CONCEPT.csv', delimiter='\t', low_memory=False)
concept_relationships = pd.read_csv(CDM_PATH / 'CONCEPT_RELATIONSHIP.csv', delimiter='\t', low_memory=False)
concept_ancestors = pd.read_csv(CDM_PATH / 'CONCEPT_ANCESTOR.csv', delimiter='\t', low_memory=False)
vocabularies = pd.read_csv(CDM_PATH / 'VOCABULARY.csv', delimiter='\t', low_memory=False)
concept_class = pd.read_csv(CDM_PATH / 'CONCEPT_CLASS.csv', delimiter='\t', low_memory=False)
domain = pd.read_csv(CDM_PATH / 'DOMAIN.csv', delimiter='\t', low_memory=False)
relationship = pd.read_csv(CDM_PATH / 'RELATIONSHIP.csv', delimiter='\t', low_memory=False)

In [7]:
import omop_alchemy as oa

engine = oa.oa_config.engine
oa.Base.metadata.create_all(engine)

In [8]:
concepts_with_ids.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN


In [44]:
concept_stage

,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
rownames,,,,,,,,,
26485,Eluvixtamab,drug,HemOnc,Component,,1,2019-05-27,2099-12-31,
16860,Capivasertib,drug,HemOnc,Component,,2,2019-05-27,2099-12-31,
7847,Abciximab,drug,HemOnc,Component,,3,2019-05-27,2099-12-31,
7913,Abemaciclib,drug,HemOnc,Component,,4,2019-05-27,2099-12-31,
7924,Abexinostat,drug,HemOnc,Component,,5,2019-05-27,2099-12-31,
...,...,...,...,...,...,...,...,...,...
401009,Dara-Pd (SC daratumumab) variant #04,regimen,HemOnc,Regimen Variant,,144942,2024-11-03,2099-12-31,
411008,Dara-Pd (SC daratumumab) variant #05,regimen,HemOnc,Regimen Variant,,144943,2024-11-03,2099-12-31,
421007,Dara-Pd (SC daratumumab) variant #06,regimen,HemOnc,Regimen Variant,,144944,2024-11-03,2099-12-31,


In [9]:
# at this stage, these are the classes that exist in HemOnc but aren't pulled into OMOP

non_omop_class = ['Author', 'Reference', 'Study', 'PubMedURL', 'ReferenceDOI', 'Clinical trial ID', 'Regimen Variant', 
                  'ReferenceURL', 'PubMedCentralURL', 'City', 'Cycle Sigs', 'Regimen Stub', 'Study Group', 'Journal', 
                  'Endpoint', 'Duration', 'Year', 'Numeric', 'Endpoint Type', 'Experimental design', 'Study Class']

cc = concept_stage[~concept_stage.concept_class_id.isin(non_omop_class)].merge(concepts_with_ids, on='concept_code', how='left')

In [1]:
# how many of the HemOnc concepts that are in OMOP classes do not currently have an assigned concept_id?

len(cc[cc.concept_id.isna()])/len(cc)

NameError: name 'cc' is not defined

In [11]:
# current hemonc concepts from which classes are missing IDs from Athena?

cc[cc.concept_id.isna()].concept_class_id_x.value_counts()

concept_class_id_x
Regimen            1223
Regimen Class       421
Brand Name          160
Component            67
Component Class      46
Unit                 12
Condition             8
Frequency             6
Context               4
Component Role        3
Name: count, dtype: int64

In [12]:
# trying to clean up NDC concept codes but it doesn't really work

concept_relationship_stage.concept_code_2 = (
    concept_relationship_stage.apply(
        lambda x: x.concept_code_2.replace('-', '') if x.vocabulary_id_2 == "NDC" else x.concept_code_2, 
        axis=1
        )
)

cr = (
    concept_relationship_stage[concept_relationship_stage.vocabulary_id_2 != 'HemOnc']
    .merge(concepts_with_ids, 
           left_on=['concept_code_2', 'vocabulary_id_2'], 
           right_on=['concept_code', 'vocabulary_id'], 
           how='left')
)

In [14]:
# which non-hemonc vocabs do we need to have available for allowing this to actually extend the CDM?
concept_relationship_stage.vocabulary_id_2.value_counts()

vocabulary_id_2
HemOnc              409601
RxNorm               19074
NDC                  14588
ICD-O-3               1618
ICD-10-CM             1430
ICD-9-CM              1021
NCIT                   732
HCPCS                  461
RxNorm Extension       401
SEER Site Recode       166
OncoTree               140
Name: count, dtype: int64

In [15]:
concept_relationship_stage[concept_relationship_stage.vocabulary_id_2=='NDC']

,concept_code_1,concept_code_2,vocabulary_id_1,vocabulary_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
rownames,,,,,,,,
1100235,4,00024483,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
285916,4,00024815,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
310209,4,00025337,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
410201,4,00026216,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
510173,6,00931125,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
...,...,...,...,...,...,...,...,...
1296910,93818,50242103,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
1297010,93818,50242105,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,
1396710,94609,25682022,HemOnc,NDC,Maps to,2022-02-05,2099-12-31,


In [16]:
concepts_with_ids.vocabulary_id.value_counts()

vocabulary_id
RxNorm Extension        2146945
SNOMED                  1084286
RxNorm                   309670
OSM                      203339
ICD10CM                   99421
ICDO3                     64471
NAACCR                    34473
ICD9CM                    17564
ICD10                     16638
HemOnc                     8028
Cancer Modifier            6043
ICD9Proc                   4657
OMOP Extension             1457
UCUM                       1127
CDM                        1060
Relationship                718
Concept Class               423
UB04 Typ bill               298
SOPT                        168
Vocabulary                  149
Medicare Specialty          120
Condition Type              118
ABMS                         98
Procedure Type               97
Type Concept                 80
Domain                       65
CMS Place of Service         63
UB04 Pt dis status           55
Race                         53
Cost                         51
Observation Type          

In [18]:
# not sure how much this matters, but there seems to be some issues with hemonc mappings to NDC - codes are a bit of a mish-mash with dropped hyphens and leading 0s removed
cr[cr.concept_id.isna()].vocabulary_id_2.value_counts()

vocabulary_id_2
NDC                 14588
ICD-O-3              1618
ICD-10-CM            1430
ICD-9-CM             1021
NCIT                  732
HCPCS                 461
SEER Site Recode      166
OncoTree              140
RxNorm                  7
Name: count, dtype: int64

In [19]:
code_mods = concept_stage.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id', 'concept_name']
    ], 
    on=['concept_code', 'vocabulary_id'],
    how='left'
)

In [43]:
code_mods[(code_mods.concept_name_x != code_mods.concept_name_y) & ~code_mods.concept_class_id.isin(non_omop_class) & code_mods.concept_name_y.notna()]

,concept_name_x,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id,concept_name_y
0,Eluvixtamab,drug,HemOnc,Component,,1,2019-05-27,2099-12-31,,35802855.0,AMG 330
55,BHQ-880,drug,HemOnc,Component,,57,2019-05-27,2099-12-31,,35802910.0,BHQ880
57,BCG vaccine,drug,HemOnc,Component,,59,2019-05-27,2099-12-31,,35802912.0,Bacillus Calmette-Guerin
208,Andexanet alfa,drug,HemOnc,Component,,211,2019-05-27,2099-12-31,,35803063.0,Factor Xa recombinant inactivated-zhzo
226,Leucovorin,drug,HemOnc,Component,,229,2019-05-27,2099-12-31,,35803081.0,Folinic acid
...,...,...,...,...,...,...,...,...,...,...,...
60755,HPV Oropharyngeal cancer,condition,HemOnc,Condition,,83479,2019-07-22,2099-12-31,,42542448.0,HPV-positive Oropharyngeal cancer
61307,Isotretinoin and Dinutuximab beta,regimen,HemOnc,Regimen,S,84292,2019-07-22,2099-12-31,,35101312.0,Isotretinoin and Dinutuximab
67085,Nystatin,drug,HemOnc,Component,,91138,2019-07-22,2099-12-31,,35101066.0,Mycostatin
67777,Mycostatin,drug,HemOnc,Brand Name,,91912,2019-07-22,2099-12-31,,35101023.0,Nystatin


In [20]:
# there are some hemonc components that have changed concept_code? this will not be backwards compatible?
code_mods[(code_mods.concept_name_x != code_mods.concept_name_y) & ~code_mods.concept_class_id.isin(non_omop_class) & code_mods.concept_name_y.notna()].to_csv('reused_codes.csv', index=False)

In [21]:
vocab_id_lookup = {
    'ICD-10-CM': 'ICD10CM',
    'ICD-9-CM': 'ICD9CM',
    'ICD-O-3': 'ICDO3'
}

concept_relationship_stage['vocabulary_id'] = concept_relationship_stage.vocabulary_id_2.map(lambda x: vocab_id_lookup[x] if x in vocab_id_lookup else x)

In [25]:
# get all the OMOP concept IDs that exist for the current OMOP-ready hemonc codes

concept_stage_id = concept_stage.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id', 'concept_name'] # note that merging on concept_name shouldn't be required here, but as per above with code mods, some of the concept codes in hemonc have changed over time
    ], 
    how='left'
)

# also for the hemonc concepts in the relationship file

concept_relationship_stage_id = concept_relationship_stage.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id']
    ].rename(
        columns={'concept_code': 'concept_code_1', 
                 'concept_id': 'concept_id_1', 
                 'vocabulary_id': 'vocabulary_id_1'}
        ), 
    how='left'
)

# and then the related concepts in the relationship file

concept_relationship_stage_id = concept_relationship_stage_id.merge(
    concepts_with_ids[
        ['concept_code', 'concept_id', 'vocabulary_id']
    ].rename(
        columns={'concept_code': 'concept_code_2', 
                 'concept_id': 'concept_id_2'}
        ), 
    how='left'
)

In [22]:
# we need to grab all the concepts specifically for vocab, relationships, domains and concept classes so we can insert those as bulk initial load and avoid referential integrity issues

vocab_concepts = vocabularies.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'vocabulary_id']], 
    left_on='vocabulary_concept_id', 
    right_on='concept_id'
)

rel_concepts = relationship.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'relationship_id']], 
    left_on='relationship_concept_id', 
    right_on='concept_id'
)

domain_concepts = domain.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'domain_id']], 
    left_on='domain_concept_id', 
    right_on='concept_id'
)

concept_class_concepts = concept_class.merge(
    concepts_with_ids[[c for c in concepts_with_ids.columns if c != 'concept_class_id']], 
    left_on='concept_class_concept_id', 
    right_on='concept_id'
)

In [27]:
concept_relationship_stage_id.to_csv('ho_gen_rel_stage_with_ids.csv', index=False)

In [28]:
concept_relationship_stage_id[concept_relationship_stage_id.concept_id_1==1525049]

,concept_code_1,concept_code_2,vocabulary_id_1,vocabulary_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason,vocabulary_id,concept_id_1,concept_id_2
272800,52958,642,HemOnc,HemOnc,Is a,2022-03-28,2099-12-31,,HemOnc,1525049.0,42542211.0
272801,52958,22030,HemOnc,SEER Site Recode,Maps to,2023-11-19,2099-12-31,,SEER Site Recode,1525049.0,NaN
272802,52958,C34.0,HemOnc,ICD-O-3,Maps to,2024-03-17,2099-12-31,,ICDO3,1525049.0,44498182.0
272803,52958,C34.1,HemOnc,ICD-O-3,Maps to,2024-03-17,2099-12-31,,ICDO3,1525049.0,44498183.0
272804,52958,C34.2,HemOnc,ICD-O-3,Maps to,2024-03-17,2099-12-31,,ICDO3,1525049.0,44498184.0
272805,52958,C34.3,HemOnc,ICD-O-3,Maps to,2024-03-17,2099-12-31,,ICDO3,1525049.0,44498185.0
272806,52958,C34.8,HemOnc,ICD-O-3,Maps to,2024-03-17,2099-12-31,,ICDO3,1525049.0,44498186.0
272807,52958,C34.9,HemOnc,ICD-O-3,Maps to,2024-03-17,2099-12-31,,ICDO3,1525049.0,44498187.0


In [29]:
# this is where we are building in the compatibility with the OMOP Alchemy classes

from omop_alchemy.model.vocabulary import Concept, Vocabulary, Domain, Concept_Class, Relationship, Concept_Relationship, Concept_Ancestor

def make_concept(concept_row):
    return Concept(concept_id = int(concept_row.concept_id),
                   concept_name = concept_row.concept_name if not pd.isna(concept_row.concept_name) else "",
                   domain_id = concept_row.domain_id,
                   vocabulary_id = concept_row.vocabulary_id,
                   concept_class_id = concept_row.concept_class_id,
                   standard_concept = concept_row.standard_concept if not pd.isna(concept_row.standard_concept) else None,
                   concept_code = concept_row.concept_code,
                   valid_start_date = datetime.strptime(str(concept_row.valid_start_date), '%Y%m%d'), 
                   valid_end_date = datetime.strptime(str(concept_row.valid_end_date), '%Y%m%d'), 
                   invalid_reason = concept_row.invalid_reason if not pd.isna(concept_row.invalid_reason) else None)

def make_vocab(vocab_row):
    v = Vocabulary(vocabulary_id = vocab_row.vocabulary_id,
                   vocabulary_name = vocab_row.vocabulary_name,
                   vocabulary_reference = vocab_row.vocabulary_reference,
                   vocabulary_concept_id = vocab_row.vocabulary_concept_id, 
                   vocabulary_version = vocab_row.vocabulary_version)
    v.vocabulary_concept = vocab_row.concept_object
    return v
    
def make_domain(domain_row):
    d = Domain(domain_id = domain_row.domain_id,
               domain_name = domain_row.domain_name,
               domain_concept_id = domain_row.domain_concept_id)
    d.domain_concept = domain_row.concept_object
    return d

def make_concept_class(cc_row):
    c = Concept_Class(concept_class_id = cc_row.concept_class_id,
                      concept_class_name = cc_row.concept_class_name,
                      concept_class_concept_id = cc_row.concept_class_concept_id)
    c.concept_class_concept = cc_row.concept_object
    return c


def make_rel(r_row):
    r = Relationship(relationship_id = r_row.relationship_id,
                      relationship_name = r_row.relationship_name,
                      is_hierarchical = r_row.is_hierarchical,
                      defines_ancestry = r_row.defines_ancestry,
                      reverse_relationship_id = r_row.reverse_relationship_id,
                      relationship_concept_id = r_row.relationship_concept_id)
    r.relationship_concept = r_row.concept_object
    return r

In [30]:
# preserve referential integrity by adding all the concept class, vocabulary and domain concepts in one go
meta_concepts = pd.concat([concept_class_concepts.concept_id, domain_concepts.concept_id, vocab_concepts.concept_id, rel_concepts.concept_id])
meta_concepts_df = concepts_with_ids.merge(meta_concepts)
meta_concepts_df['concept_object'] = meta_concepts_df.apply(make_concept, axis=1)


concept_class_concepts = concept_class_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='concept_class_concept_id', right_on='concept_id')
domain_concepts = domain_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='domain_concept_id', right_on='concept_id')
vocab_concepts = vocab_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='vocabulary_concept_id', right_on='concept_id')
rel_concepts = rel_concepts.merge(meta_concepts_df[['concept_id', 'concept_object']], left_on='relationship_concept_id', right_on='concept_id')

In [31]:
# full set of hemonc relationships - some obviously do not need to be pulled in - TBC how to cut down

concept_relationship_stage.relationship_id.value_counts()

relationship_id
Has middle author       106508
Is a                     62847
Was studied in           36946
Maps to                  20926
Has modality             11824
                         ...  
Has anticoag Rx             21
Was NMPA approved yr        19
Has radiotherapy Rx          9
Has pept-drug cjgt           1
Has PDC Rx                   1
Name: count, Length: 89, dtype: int64

In [32]:
# if you're pulling in only hemonc concepts that have a valid domain_id, then you can retain only relationships that have concepts in allocated domains on both sides
# this is kind of circular logic, but an ok place to start

concept_stage_id.domain_id.value_counts()

domain_id
               93217
regimen         8032
drug            5892
measurement      778
condition        282
procedure        110
Name: count, dtype: int64

In [33]:
domain_lookup_by_code = {k:v for k, v in zip(concept_stage_id.concept_code, concept_stage_id.domain_id)}
domain_lookup_by_id = {k:v for k, v in zip(concepts_with_ids.concept_id, concepts_with_ids.domain_id)}

In [34]:
# try find all known domain_id values either from hemonc source or existing OMOP 

concept_rels_with_domains = concept_relationship_stage_id[concept_relationship_stage_id.invalid_reason != 'D'].copy()
concept_rels_with_domains['domain_id_1'] = concept_rels_with_domains.concept_code_1.map(domain_lookup_by_code)
concept_rels_with_domains['domain_id_2'] = concept_rels_with_domains.concept_code_2.map(domain_lookup_by_code)
concept_rels_with_domains.loc[concept_rels_with_domains['domain_id_1'].isna(), 'domain_id_1'] = concept_rels_with_domains.concept_id_1.map(domain_lookup_by_id)
concept_rels_with_domains.loc[concept_rels_with_domains['domain_id_2'].isna(), 'domain_id_2'] = concept_rels_with_domains.concept_id_2.map(domain_lookup_by_id)

In [35]:
# that would imply that these are the relationship types that you need to exist before import
concept_rels_with_domains[
    (concept_rels_with_domains.domain_id_1!='') & 
    (concept_rels_with_domains.domain_id_2!='') 
    ].relationship_id.value_counts()

relationship_id
Is a                   61061
Maps to                20593
Has cytotoxic chemo    10340
Has context             7100
Is current in adult     6574
                       ...  
Has anticoag Rx           14
Has antineopl Rx          14
Has radiotherapy Rx        8
Has pept-drug cjgt         1
Has PDC Rx                 1
Name: count, Length: 64, dtype: int64

In [36]:
target_rels = concept_rels_with_domains[(concept_rels_with_domains.domain_id_1!='') & (concept_rels_with_domains.domain_id_2!='') ].relationship_id.value_counts().reset_index()
target_rels = target_rels.merge(relationship, how='left')

In [37]:
# means these may be the new relationship types being sought - confirmation required with respect to the 'year' concept class?
target_rels[target_rels.relationship_name.isna()]

,relationship_id,count,relationship_name,is_hierarchical,defines_ancestry,reverse_relationship_id,relationship_concept_id
4,Is current in adult,6574,NaN,NaN,NaN,NaN,NaN
8,Has been compared to,4247,NaN,NaN,NaN,NaN,NaN
10,Has synthetic regimen,2791,NaN,NaN,NaN,NaN,NaN
14,Can be preceded by,1079,NaN,NaN,NaN,NaN,NaN
16,Can be followed by,985,NaN,NaN,NaN,NaN,NaN
20,Is historical in adult,606,NaN,NaN,NaN,NaN,NaN
21,Was FDA approved yr,544,NaN,NaN,NaN,NaN,NaN
26,Has PMDA indication,339,NaN,NaN,NaN,NaN,NaN
28,Was EMA approved yr,308,NaN,NaN,NaN,NaN,NaN
33,Has minor class,226,NaN,NaN,NaN,NaN,NaN


In [38]:
# query why one of the vocabularies has a null ID from athena?

vocab_concepts[vocab_concepts.vocabulary_id.isna()]

,vocabulary_id,vocabulary_name,vocabulary_reference,vocabulary_version,vocabulary_concept_id,concept_id_x,concept_name,domain_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id_y,concept_object
14,NaN,OMOP Standardized Vocabularies,OMOP generated,v5.0 30-AUG-24,44819096,44819096,OMOP Standardized Vocabularies,Metadata,Vocabulary,NaN,OMOP generated,19700101,20991231,NaN,44819096,<Concept 44819096 - OMOP generated (OMOP Stand...


In [39]:
vocab_concepts.vocabulary_id = vocab_concepts.vocabulary_id.fillna('empty') 

In [40]:
def make_ancestor(a_row):
    return Concept_Ancestor(ancestor_concept_id = int(a_row.ancestor_concept_id),
                            descendant_concept_id = int(a_row.descendant_concept_id))
                            #min_levels_of_separation = a_row.min_levels_of_separation,
                            #max_levels_of_separation = a_row.max_levels_of_separation)
                    

In [41]:
# create metadata concepts & make objects for other reference tables
concept_class_concepts['ob'] = concept_class_concepts.apply(make_concept_class, axis=1)
domain_concepts['ob'] = domain_concepts.apply(make_domain, axis=1)
vocab_concepts['ob'] = vocab_concepts.apply(make_vocab, axis=1)
rel_concepts['ob'] = rel_concepts.apply(make_rel, axis=1)

all_ob = pd.concat([concept_class_concepts[['concept_id_x', 'ob']], 
                    domain_concepts[['concept_id_x', 'ob']], 
                    vocab_concepts[['concept_id_x', 'ob']], 
                    rel_concepts[['concept_id_x', 'ob']]])

In [42]:

with so.Session(engine) as sess:
    sess.add_all(list(concept_class_concepts.ob))
    sess.add_all(list(domain_concepts.ob))
    sess.add_all(list(vocab_concepts.ob))
    sess.add_all(list(rel_concepts.ob))
    sess.add_all(list(meta_concepts_df.concept_object))
    sess.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: concept.concept_id
[SQL: INSERT INTO concept (concept_id, concept_name, concept_code, domain_id, vocabulary_id, concept_class_id, standard_concept, valid_start_date, valid_end_date, invalid_reason) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [(44819021, 'Qualifier Value', 'OMOP generated', 'Metadata', 'Concept Class', 'Concept Class', None, '1970-01-01', '2099-12-31', None), (44819092, 'Quality Metric', 'OMOP generated', 'Metadata', 'Concept Class', 'Concept Class', None, '1970-01-01', '2099-12-31', None), (44819030, 'Race', 'OMOP generated', 'Metadata', 'Concept Class', 'Concept Class', None, '1970-01-01', '2099-12-31', None), (44819273, 'Read', 'OMOP generated', 'Metadata', 'Concept Class', 'Concept Class', None, '1970-01-01', '2099-12-31', None), (44818999, 'Record Artifact', 'OMOP generated', 'Metadata', 'Concept Class', 'Concept Class', None, '1970-01-01', '2099-12-31', None), (44819274, 'OMOP Relationship'

In [36]:
with so.Session(engine) as sess:
    existing_vocabs = pd.DataFrame(sess.query(Concept.vocabulary_id).distinct().all())
    existing_concepts = pd.DataFrame(sess.query(Concept.concept_id).distinct().all())

existing_vocabs.head()

,vocabulary_id
0,Domain
1,Concept Class
2,Relationship
3,Vocabulary


In [37]:
concept_stage_id[~concept_stage_id.concept_class_id.isin(non_omop_class) & 
                 ~concept_stage_id.vocabulary_id.isin(['Concept Class', 'Domain', 'Vocabulary', 'Relationship'])  
#                 &~concept_stage_id.concept_id.isin(existing_concepts.concept_id.unique())
                 ].concept_class_id.value_counts()

concept_class_id
Regimen            5566
Brand Name         4586
Component           804
Regimen Class       752
Component Class     488
Condition           282
Procedure            63
Context              47
Unit                 37
Modality             23
Route                14
Frequency            14
Component Role        3
Null                  1
Name: count, dtype: int64

In [38]:
concepts_to_add = concept_relationship_stage_id[
    ['concept_code_2', 'vocabulary_id']
    ].drop_duplicates().merge(
        concepts_with_ids, 
        left_on=['concept_code_2', 'vocabulary_id'], 
        right_on=['concept_code', 'vocabulary_id']
    )

In [39]:
len(concepts_to_add[~concepts_to_add.concept_id.isin(existing_concepts.concept_id.unique())])

9570

In [40]:
# add in concepts both from within the HemOnc universe, as well as all available cross-mappings
ho_conc = concepts_with_ids[concepts_with_ids.vocabulary_id=='HemOnc'].apply(make_concept, axis=1)
non_ho_conc = concepts_to_add[~concepts_to_add.vocabulary_id.isin(['Concept Class', 'Domain', 'Vocabulary', 'Relationship', 'HemOnc'])].apply(make_concept, axis=1)

In [41]:
with so.Session(engine) as sess:
    sess.add_all(ho_conc)
    sess.add_all(non_ho_conc)
    sess.commit()

In [42]:
# uncomment this section only if you want to pull in extended vocab, not just those pointed to by hemonc relationships

# use this one if you want completeness for vocabs that are involved in hemonc relationships, even the concepts that aren't pointed 
# to directly by those relationships
# target_vocabularies = concept_relationship_stage.vocabulary_id.unique()

# use this one if you want useful oncology-extension relevant vocabs
target_vocabularies = ['Cancer Modifier', 'OMOP Extension', 'NAACCR', 'Gender', 'Episode Type', 
                       'Visit', 'Condition Type', 'Procedure Type', 'Type Concept', 'Observation Type',
                       'Visit Type', 'Drug Type', 'Death Type', 'Meas Type', 'Note Type',
                       'Cost Type', 'Obs Period Type', 'Episode Type', 'Device Type', 'Domain', 
                       'Ethnicity', 'Language', 'Relationship', 'Race', 'CDM', 'Concept Class',
                       'RxNorm', 'ICD10CM', 'ICD03', 'Cancer Modifier']

# you can handle a larger chunksize if using non-sqlite backend - 50k is large but reasonable on a local device if sqlite

chunksize = 50000

for vocab in target_vocabularies:
    v = concepts_with_ids[concepts_with_ids.vocabulary_id==vocab]
    if len(v) > 0:
        with so.Session(engine) as sess:
            existing = pd.DataFrame(sess.query(Concept.concept_id).filter(Concept.vocabulary_id==vocab))
            if len(existing) > 0:
                v = v.merge(existing, how="left", indicator=True)
                v = v[v._merge=='left_only']
            for i in range(0, len(v), chunksize):
                to_add_objects = v[i:i+chunksize].apply(make_concept, axis=1)
                sess.add_all(to_add_objects)
                sess.commit()
    print(vocab, len(v), len(existing), len(v))



Cancer Modifier 6043 0 6043
OMOP Extension 1457 0 1457
NAACCR 34473 0 34473
Gender 5 0 5
Episode Type 5 0 5
Visit 20 0 20
Condition Type 118 0 118
Procedure Type 97 0 97
Type Concept 80 0 80
Observation Type 29 0 29
Visit Type 18 0 18
Drug Type 16 0 16
Death Type 14 0 14
Meas Type 12 0 12
Note Type 10 0 10
Cost Type 8 0 8
Obs Period Type 6 0 6
Episode Type 0 5 0
Device Type 4 0 4
Domain 15 50 15
Ethnicity 2 0 2
Language 1 0 1
Relationship 22 696 22
Race 53 0 53
CDM 1060 0 1060
Concept Class 0 423 0
RxNorm 309085 585 309085
ICD10CM 98423 998 98423
ICD03 0 998 0
Cancer Modifier 0 6043 0


In [43]:
with so.Session(engine) as sess:
    existing_vocabs = pd.DataFrame(sess.query(Concept.vocabulary_id).distinct().all())
    existing_concepts = pd.DataFrame(sess.query(Concept.concept_id).distinct().all())

existing_vocabs.head()

,vocabulary_id
0,Domain
1,Concept Class
2,Relationship
3,Condition Type
4,Vocabulary


In [44]:
def make_relationship(rel_row):
    return Concept_Relationship(concept_id_1 = rel_row.concept_id_1,
                         concept_id_2 = rel_row.concept_id_2,
                         relationship_id = rel_row.relationship_id)

# we cannot set relationships for any new concepts that do not have concept_id assigned, so note that these will be missing

ho_rels = concept_relationship_stage_id[
    concept_relationship_stage_id.concept_id_1.notna() &
    concept_relationship_stage_id.concept_id_2.notna()
    ].drop_duplicates(
        subset=['concept_id_1', 'concept_id_2', 'relationship_id']
        ).apply(make_relationship, axis=1)

In [45]:
ancestors = concept_ancestors.merge(
    existing_concepts[
        ['concept_id']
        ].rename(columns={'concept_id': 'ancestor_concept_id'})
    ).merge(
        existing_concepts[
            ['concept_id']
        ].rename(columns={'concept_id': 'descendant_concept_id'})
    ).apply(make_ancestor, axis=1)  

In [46]:
len(ancestors), len(ho_rels)

(1363480, 52023)

In [47]:
with so.Session(engine) as sess:
    for i in range(0, len(ancestors), 50000):
        sess.add_all(list(ancestors[i:i+50000]))
        sess.commit()

In [48]:
with so.Session(engine) as sess:
    sess.add_all(ho_rels)
    sess.commit()

In [49]:
# uncomment this section if you want to add extended relationships that involve only non-hemonc concepts for specific extra vocabs
# don't need to do the same for ancestry, as we don't filter that by vocab - it's done by existence of concepts already above, so if you pulled
# them in, the ancestry will be there

chunksize = 50000

# get only relationships where both concepts have been imported

target_relationships = (
    concept_relationships
    .merge(existing_concepts.rename(columns={'concept_id': 'concept_id_1'}))
    .merge(existing_concepts.rename(columns={'concept_id': 'concept_id_2'}))
)

with so.Session(engine) as sess:
    existing_relationships = pd.DataFrame(
        sess.query(
            Concept_Relationship.concept_id_1, Concept_Relationship.concept_id_2, Concept_Relationship.relationship_id
            )
    )

relationships_to_add = target_relationships.merge(existing_relationships, how='left', indicator=True)
relationships_to_add = relationships_to_add[relationships_to_add._merge=='left_only']

with so.Session(engine) as sess:
    for i in range(0, len(relationships_to_add), chunksize):
        to_add = relationships_to_add[i:i+chunksize].apply(make_relationship, axis=1)
        sess.add_all(list(to_add))
        sess.commit()

In [50]:
# due to misalignment with current athena version we have to accept gap in concept_id completeness even for classes that should have them

concept_stage_id[concept_stage_id.concept_id.isna() & ~concept_stage_id.concept_class_id.isin(non_omop_class)].head()

,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id
0,Eluvixtamab,drug,HemOnc,Component,,1,2019-05-27,2099-12-31,,NaN
55,BHQ-880,drug,HemOnc,Component,,57,2019-05-27,2099-12-31,,NaN
57,BCG vaccine,drug,HemOnc,Component,,59,2019-05-27,2099-12-31,,NaN
208,Andexanet alfa,drug,HemOnc,Component,,211,2019-05-27,2099-12-31,,NaN
226,Leucovorin,drug,HemOnc,Component,,229,2019-05-27,2099-12-31,,NaN


In [53]:
# helper mapping data / merges within OMOP concepts
is_a = concept_relationship_stage_id[
    concept_relationship_stage_id.relationship_id=='Is a'
    ][['concept_code_1', 'concept_code_2', 'concept_id_1', 'concept_id_2']]


brand_mappings = concept_stage_id[
    concept_stage_id.concept_class_id=='Brand Name'
    ].merge(
        concept_relationship_stage_id[
            concept_relationship_stage_id.relationship_id=='Has brand name'
        ][['concept_code_1', 'concept_code_2', 'concept_id_1', 'concept_id_2']], 
        left_on='concept_code', 
        right_on='concept_code_2', 
        how='left'
    )

component_df = concept_stage_id[
    concept_stage_id.concept_class_id.isin(['Component', 'Procedure'])
    ].copy()

component_class_df = concept_stage_id[
    concept_stage_id.concept_class_id=='Component Class'
    ][['concept_name','concept_code','concept_id']].rename(
        columns={'concept_code': 'concept_code_2', 
                 'concept_name': 'concept_class_name'}
    )

has_modality = concept_relationship_stage_id[
    concept_relationship_stage_id.relationship_id=='Has modality'
    ][['concept_code_1', 'concept_code_2', 'concept_id_1', 'concept_id_1']]

modality_df = concept_stage_id[
    concept_stage_id.concept_class_id=='Modality'
    ][['concept_name','concept_code','concept_id']].rename(
        columns={'concept_code': 'concept_code_2', 
                 'concept_name': 'modality_name', 
                 'concept_id': 'concept_id_2'}
        )

In [51]:
route_mappings = {'44954': '26643006', 
                  '44957': '47625008',
                  '44979': '37161004',
                  '44994': '6064005',
                  '45080': '78421000',
                  '45153': '34206005',
                  '45215': '372471009',
                  '45273': '420254004',
                  '45426': '372466002',
                  '45531': '46713006',
                  '45574': '72607000',
                  '45684': '58100008',
                  '45939': '447694001'}

In [54]:
# helper data cleaning / transformation functions

def get_enum(e, s):
    try:
        return e[s.lower().strip().replace('-', '_').replace(' ', '_').replace('/', '_')]
    except:
        return None

def get_date(y, m, d):
    try:    
        datetime.date(int(y), int(m), int(d))
    except:
        return None

In [96]:
# functions to take dataframe rows and return database objects

def make_context(context_row):
    return Hemonc_Context(context_code = context_row.contextRaw,
                          context_name = context_row.contextPretty,
                          intent = get_enum(Intent, context_row.intent),
                          setting = get_enum(Setting, context_row.intent),
                          risk_stratification = get_enum(Risk, context_row.intent),
                          phenotype = get_enum(Phenotype, context_row.intent),
                          prior_therapy = get_enum(PriorTherapy, context_row.prior_therapy),
                          date_added = context_row.date_added)


def make_study(study_row):
    enrollment_from = get_date(study_row.from_year, study_row.from_month, study_row.from_day)
    enrol_to = get_date(study_row.to_year, study_row.to_month, study_row.to_day)
    return Hemonc_Study(study_code = study_row.study,
                        registry = study_row.registry,
                        trial_id = study_row.trial_id,
                        condition_code = study_row.concept_code,
                        enrollment_from = enrollment_from if not pd.isna(enrollment_from) else None,
                        enrollment_to = enrol_to if not pd.isna(enrol_to) else None,
                        phase = study_row.phase,
                        study_design = get_enum(StudyDesign, study_row.study_design),
                        study_design_imputed = study_row.study_design_imputed,
                        sact = study_row.sact if not pd.isna(study_row.sact) else None,
                        protocol = study_row.protocol,
                        fda_reg_study = study_row.fda_reg_study,
                        fda_unreg_study = study_row.fda_unreg_study,
                        start = study_row.start  if not pd.isna(study_row.start) else None,
                        end = study_row.end  if not pd.isna(study_row.end) else None,
                        study_group = study_row.study_group,
                        sponsor = study_row.sponsor,
                        date_added = study_row.date_added if not pd.isna(study_row.date_added) else None, 
                        date_modified = study_row.date_last_modified if not pd.isna(study_row.date_last_modified) else None
                        )

def make_regimen(regimen_row):
    return Hemonc_Regimen(regimen_cui = regimen_row.regimen_cui,
                          regimen_name = regimen_row.regimen,
                          regimen_concept_id = int(regimen_row.concept_id) if not pd.isna(regimen_row.concept_id) else None)

def make_condition(condition_row):
    return Hemonc_Condition(condition_code = condition_row.concept_code,
                            condition_name = condition_row.concept_name,
                            condition_concept_id = int(condition_row.concept_id) if not pd.isna(condition_row.concept_id) else None)


def make_component(component_class_row):
    return Hemonc_Component(component_code = component_class_row.concept_code,
                            component_name = component_class_row.concept_name,
                            component_concept_id = int(component_class_row.concept_id) if not pd.isna(component_class_row.concept_id) else None)

def make_component_role(role_row):
    return Hemonc_Component_Role(regimen_cui = role_row.concept_code_1,
                                 component_code = role_row.concept_code_2 if role_row.component_class in ['Component', 'Procedure'] else None,
                                 component_class_code = role_row.concept_code_2 if role_row.component_class == 'Component Class' else None,
                                 relationship_id = role_row.relationship_id)


def make_component_class(component_class_row):
    return Hemonc_Component_Class(component_class_code = component_class_row.concept_code_2,
                                  component_class_name = component_class_row.concept_class_name,
                                  component_class_concept_id = int(component_class_row.concept_id) if not pd.isna(component_class_row.concept_id) else None)

def make_branch(branch_row):
    return Hemonc_Branch_Conditional(branch_name = branch_row.original,
                                     branch_type = branch_row.RULE_TYPE if not pd.isna(branch_row.RULE_TYPE) else BranchConditionalType.other,
                                     numeric_min = branch_row.MIN_NUM,
                                     numeric_max = branch_row.MAX_NUM,
                                     value = branch_row.RULE_VALUE)

def make_variant(variant_row):
    return Hemonc_Variant(variant_cui = variant_row.variant_cui,
                         variant_name = variant_row.variant,
                         regimen_cui = variant_row.regimen_cui)


def make_reg_part(reg_part_row):
    return Hemonc_Regimen_Part(variant_cui = reg_part_row.variant_cui,
                               regimen_part_id = reg_part_row.regimen_part_id,
                               portion = reg_part_row.portion if not pd.isna(reg_part_row.portion) else None,
                               cycle_sig_id = reg_part_row.cyclesigs,
                               timing = reg_part_row.timing,
                               timing_unit = reg_part_row.timing_unit
                               )

def assign_part_phase(reg_part_row):
    if not pd.isna(reg_part_row.phase):
        phases = [Part_Phase(regimen_part_id = reg_part_row.regimen_part_id,
                             variant_cui = reg_part_row.variant_cui, 
                             phase = get_enum(Phase, reg_part_row.phase),
                             ) for phase in reg_part_row.phase.split('|')]
        return phases

def make_cycle_sig(cs_row):
    return Hemonc_Cycle_Sig(cycle_sig_id = cs_row.cyclesigs,
                            duration_min = cs_row.DUR_MIN, 
                            duration_max = cs_row.DUR_MAX,
                            duration_units = cs_row.DUR_UNITS,
                            frequency_min = cs_row.FREQ_MIN,
                            frequency_max = cs_row.FREQ_MAX,
                            frequency_units = cs_row.FREQ_UNITS,
                            repeats_min = cs_row.REP_MIN,
                            repeats_max = cs_row.REP_MAX,
                            repeats_units = cs_row.REP_UNITS, 
                            cycle_len_min = cs_row.cycle_length_lb if not pd.isna(cs_row.cycle_length_lb) else None,
                            cycle_len_max = cs_row.cycle_length_ub if not pd.isna(cs_row.cycle_length_ub) else None,
                            cycle_len_units = cs_row.cycle_length_unit if not pd.isna(cs_row.cycle_length_unit) else None,
                            residual = ' '.join([t.text for t in cs_row.cs_residual]))


def make_modality(mod_row):
    return Hemonc_Modality(modality_code = mod_row.concept_code_2,
                           modality_name = mod_row.modality_name,
                           modality_concept_id = int(mod_row.concept_id_2) if not pd.isna(mod_row.concept_id_2) else None)

def make_ref(ref_row):
    return Hemonc_Ref(reference = ref_row.reference,
                      condition_code = ref_row.concept_code,
                      pmid = ref_row.pmid,
                      study = ref_row.study,
                      title = ref_row.title,
                      pmcid = ref_row.pmcid,
                      doi = ref_row.doi if not ref_row.doi=="" else None,
                      url = ref_row.url if not ref_row.url=="" else None,
                      journal = ref_row.journal,
                      biblio = ref_row.biblio,
                      pub_date = ref_row.pub_date,
                      order = ref_row.order,
                      update = ref_row['update'],
                      ref_type = ref_row.ref_type)

def make_sig(sig_row):
    return Hemonc_Sig(regimen_part_id = sig_row.regimen_part_id,
                      variant_cui = sig_row.variant_cui,
                      sig_id = sig_row.sig_id,
                      component_code = sig_row.concept_code,
                      component_name = sig_row.component,
                      component_role = get_enum(ComponentRole, sig_row.component_role),
                      step_number = sig_row.step_number,
                      component_class = sig_row['class'],
                      tail = sig_row['tail'],
                      route = sig_row.route,
                      doseMinNum = sig_row.doseMinNum if not pd.isna(sig_row.doseMinNum) else None,
                      doseMaxNum = sig_row.doseMaxNum if not pd.isna(sig_row.doseMaxNum) else None,
                      doseUnit = sig_row.doseUnit if not pd.isna(sig_row.doseUnit) else None,
                      doseCapNum = sig_row.doseCapNum if not pd.isna(sig_row.doseCapNum) else None,
                      doseCapUnit = sig_row.doseCapUnit if not pd.isna(sig_row.doseCapUnit) else None,
                      durationMinNum = sig_row.durationMinNum if not pd.isna(sig_row.durationMinNum) else None,
                      durationMaxNum = sig_row.durationMaxNum if not pd.isna(sig_row.durationMaxNum) else None,
                      durationUnit = sig_row.durationUnit if not pd.isna(sig_row.durationUnit) else None,
                      frequency = sig_row.frequency if not pd.isna(sig_row.frequency) else None,
                      inParens = sig_row.inParens if not pd.isna(sig_row.inParens) else None,
                      sequence = sig_row.sequence if not pd.isna(sig_row.sequence) else None,
                      seq_rel = sig_row['seq.rel'] if not pd.isna(sig_row['seq.rel']) else None,
                      seq_rel_what = sig_row['seq.rel.what'] if not pd.isna(sig_row['seq.rel.what']) else None)            


def make_sig_days(sd_row):
    days = '0' if pd.isna(sd_row.allDays) else sd_row.allDays
    return [Sig_Days(regimen_part_id = sd_row.regimen_part_id,
                     variant_cui = sd_row.variant_cui,
                     sig_id = sd_row.sig_id,
                     day = d) for d in days.split(',')]


# spacer matcher helper functions

small_num_lookup = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10}

def safe_num(tok):
    try:
        return int(tok.text)
    except:
        try:
            return float(tok.text)
        except:
            try: 
                return small_num_lookup[tok.text.lower()]
            except:
                return None

def safe_age(numeric):
    ages = [safe_num(t) for t in numeric if t._.AGE]
    if len(ages) > 0:
        return ages[0]

def get_full_stopping_condition(doc, conditions):
    return doc[conditions[0].i:] if len(conditions) > 0 else None

def get_units(tokens):
    try:
        return [t.lemma_ for t in tokens if t._.TIMING_UNIT][0]
    except:
        return None

def get_min(group):
    try:
        return min([i for i in [safe_num(t) for t in group] if i])
    except:
        return None

def get_max(group): 
    try:
        if any([t._.GTE for t in group]):
            return -1
        return max([i for i in [safe_num(t) for t in group] if i])
    except:
        return None


In [56]:
role = ['Has cytotoxic chemo', 'Has targeted therapy', 'Has supportive med',
       'Has steroid tx', 'Has local therapy', 'Has immunosuppressor',
       'Has immunotherapy', 'Has endocrine tx', 'Has radiotherapy',
       'Has growth factor', 'Has AB-drug cjgt', 'Has radioconjugate',
       'Has antineoplastic', 'Has anticoag tx', 'Has pept-drug cjgt']

concept_reg_roles = (
    concept_relationship_stage_id[concept_relationship_stage_id.relationship_id.isin(role)][['concept_code_1', 'concept_id_1', 'relationship_id', 'concept_code_2', 'concept_id_2']]
        .merge(
                concept_stage_id[['concept_code', 'concept_name', 'concept_class_id']]
                .rename(
                    columns={'concept_code': 'concept_code_1', 
                             'concept_name': 'regimen_name',
                             'concept_class_id': 'regimen_class'}
                )
        )
        .merge(
                concept_stage_id[['concept_code', 'concept_name', 'concept_class_id']]
                .rename(
                    columns={'concept_code': 'concept_code_2', 
                             'concept_name': 'component_name',
                             'concept_class_id': 'component_class'}
                )
        )  
)

In [60]:
concept_reg_roles.head()

,concept_code_1,concept_id_1,relationship_id,concept_code_2,concept_id_2,regimen_name,regimen_class,component_name,component_class,role_object
0,795,35803428.0,Has immunosuppressor,122,35802975.0,Cyclophosphamide and Prednisolone,Regimen,Cyclophosphamide,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
1,795,35803428.0,Has immunosuppressor,417,35803267.0,Cyclophosphamide and Prednisolone,Regimen,Prednisolone,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
2,797,35803429.0,Has immunosuppressor,122,35802975.0,Cyclophosphamide and Prednisone,Regimen,Cyclophosphamide,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
3,797,35803429.0,Has immunosuppressor,418,35803268.0,Cyclophosphamide and Prednisone,Regimen,Prednisone,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...
4,797,35803429.0,Has cytotoxic chemo,122,35802975.0,Cyclophosphamide and Prednisone,Regimen,Cyclophosphamide,Component,<hemonc_alchemy.model.hemonc_model.Hemonc_Comp...


In [59]:
concept_reg_roles['role_object'] = concept_reg_roles.apply(make_component_role, axis=1)

In [61]:
conditions = concept_stage_id[concept_stage_id.concept_class_id=='Condition'].apply(make_condition, axis=1)
component_class_df['component_class_object'] = component_class_df.apply(make_component_class, axis=1)

In [62]:
component_df['component_object'] = component_df.apply(make_component, axis=1)
components_with_class = component_df.merge(is_a, 
                                           left_on='concept_code', 
                                           right_on='concept_code_1').merge(component_class_df
                                                                            ).rename(columns={'concept_code_2': 'concept_class_code'})


In [ ]:
# regimen.regimen_concept_id
# hemonc_component.component_concept_id

In [63]:
modality_df['modality_object'] = modality_df.apply(make_modality, axis=1)

regimens_with_modality = concept_stage_id[
    concept_stage_id.invalid_reason == ''
    ].merge(
        has_modality, 
        left_on='concept_code', 
        right_on='concept_code_1'
    ).merge(
        modality_df
    ).rename(
            columns={'concept_code_2': 'concept_class_code'}
    ).drop_duplicates()


In [65]:
regimens_with_modality[regimens_with_modality[['concept_name', 'modality_name']].duplicated()]

,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,concept_id,concept_code_1,concept_class_code,concept_id_1,concept_id_1,modality_name,concept_id_2,modality_object


In [65]:
components_with_class.apply(lambda x: x.component_object.component_classes.append(x.component_class_object), axis=1)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
51    None
52    None
53    None
54    None
55    None
56    None
57    None
dtype: object

In [66]:
ref = ref.merge(concept_stage[concept_stage.concept_class_id=='Condition'][['concept_name', 'concept_code']], left_on='condition', right_on='concept_name', how='left')
ref['pub_date'] = pd.to_datetime(ref['pub.date'])
ref_obj = ref[~ref.concept_code.isna()].apply(make_ref, axis=1)

In [94]:
# todo: confirm what is meant by a single cui with >1 title - is this a synonym?
# probably just different titles in different pages, as this is sourced from the pointer file originally.

regimen_dedup = regimen.drop_duplicates(subset='regimen_cui').copy()

In [69]:
context.date_added = pd.to_datetime(context.date_added)
contexts = context.apply(make_context, axis=1)

In [70]:
# new issue: null condition codes - consider the fact that some of the 'conditions' being mapped to here are now actually regimen classes

# # workaround mapping
# study_condition_workaround = {
#     'Stem cell mobilization regimens': 'Stem cell mobilization regimen', 
#     'Allogeneic HSCT' : 'Allogeneic HSCT conditioning regimen',  
#     'Cellular therapy conditioning regimens' : 'Cellular therapy conditioning regimen'
# }

# study.condition = study.condition.map(lambda x: study_condition_workaround[x] if x in study_condition_workaround else x)

In [76]:
# prepare study objects
study.date_added = pd.to_datetime(study.date_added, format='mixed', errors='coerce')
study.date_last_modified = pd.to_datetime(study.date_last_modified, format='mixed', errors='coerce')
enrol_dates = study.enrollment.str.split('to', expand=True)
enrol_from = enrol_dates[0].str.strip().str.split('-', expand=True).rename(columns={0: 'from_year', 1:'from_month', 2: 'from_day'}).fillna('01')
enrol_to = enrol_dates[1].str.strip().str.split('-', expand=True).rename(columns={0: 'to_year', 1:'to_month', 2: 'to_day'}).fillna('01')
study = pd.concat([study, enrol_from, enrol_to], axis=1)
study = study.merge(concept_stage[concept_stage.concept_class_id.isin(['Condition', 'Regimen Class'])][['concept_name', 'concept_code']], left_on='condition', right_on='concept_name', how='left')
study.start = pd.to_datetime(study.start, format='mixed', errors='coerce')
study.end = pd.to_datetime(study.end, format='mixed', errors='coerce')

In [95]:
regimen_dedup = regimen_dedup.merge(regimens_with_modality[['concept_code', 'concept_id']].drop_duplicates(), left_on='regimen_cui', right_on='concept_code')

In [97]:
study['study_object'] = study.apply(make_study, axis=1)
regimen_dedup['regimen_object'] = regimen_dedup.apply(make_regimen, axis=1)

In [98]:
def append_modality(regimen_obj, modality_obj):
    if not pd.isna(modality_obj):
        regimen_obj.modalities.append(modality_obj)

In [99]:
regimen_dedup[regimen_dedup.regimen_cui.duplicated(keep=False)]

,regimen,regimen_cui,concept_code,concept_id,regimen_object


In [100]:
with so.Session(engine) as sess:
    sess.add_all(conditions)
    sess.add_all(list(component_class_df.component_class_object))
    sess.add_all(list(component_df.component_object))
    sess.add_all(contexts)
    sess.add_all(list(study[~study.concept_code.isna()].study_object))
    sess.add_all(list(modality_df.modality_object))
    sess.add_all(list(regimen_dedup.regimen_object))
    sess.add_all(list(ref_obj))
    sess.add_all(list(concept_reg_roles.role_object))
    sess.commit()

In [101]:
with so.Session(engine) as sess:
    regimen_objects = pd.DataFrame(sess.query(Hemonc_Regimen), columns=['regimen_object'])
    regimen_objects['regimen_cui'] = regimen_objects.regimen_object.map(lambda x: str(x.regimen_cui))
    modality_objects = pd.DataFrame(sess.query(Hemonc_Modality), columns=['modality_object'])
    modality_objects['modality_code'] = modality_objects.modality_object.map(lambda x: str(x.modality_code))
    modality_lookup = regimens_with_modality[['concept_code', 'concept_class_code']].drop_duplicates()
    modality_lookup = (
        modality_lookup
        .merge(regimen_objects, left_on='concept_code', right_on='regimen_cui')
        .merge(modality_objects, left_on='concept_class_code', right_on='modality_code')
    )
    modality_lookup.apply(lambda row: row.regimen_object.modalities.append(row.modality_object), axis=1)
    sess.commit()

In [102]:
# we are not going to pull in out of date variant versions
sig_vars = sigs.merge(variant[variant.version==1][['variant_cui']])

In [103]:
for (r, c), dets in sig_vars[~sig_vars.step_number.str.contains('1 of') & sig_vars.allDays.isna()].groupby(['regimen', 'component']):
    if len(dets)>1:
        break

In [104]:
# parsing out branch conditions with spacy matchers

branch_crit = sig_vars.branch.value_counts().reset_index()

all_branches = []

for crit in branch_crit.branch.unique():
    combined_branches = crit.split('AND ')
    for br in combined_branches:
        all_branches += [b.lower().strip() for b in br.split('OR ')]

all_branches = list(set(all_branches))

branch_details = pd.DataFrame({'original': all_branches, 'doc': [nlp(b) for b in all_branches]})

for label, config in rules.items():
    branch_details.doc.apply(lambda d: match_entities(matchers[label], d, label, config["MERGE"]))
    branch_details[label] = branch_details.doc.apply(lambda d: [t for t in d if getattr(t._, label)])
    branch_details[f'has_{label}'] = (branch_details[label].apply(len)>0)

branch_details['MODIFIER_HEAD'] = branch_details.doc.apply(get_modifier_child)
branch_details['NOUNS'] = branch_details.doc.apply(get_nouns)
branch_details['NUMERIC'] = branch_details.doc.map(lambda doc: [tok for tok in doc if tok.like_num])

In [105]:
# parse numeric branch factors into constituant elements

branch_details.loc[branch_details.has_AGE, 'RULE_TYPE'] = BranchConditionalType.age
branch_details.loc[branch_details.has_SIZE, 'RULE_TYPE'] = BranchConditionalType.size
branch_details.loc[branch_details.has_LAB, 'RULE_TYPE'] = BranchConditionalType.lab
branch_details.loc[branch_details.has_STAGE, 'RULE_TYPE'] = BranchConditionalType.stage

branch_details.loc[branch_details.has_STAGE, 'RULE_VALUE'] = branch_details.STAGE.apply(lambda x: ' '.join([tok.text for tok in x]) if len(x) > 0 else None)

num = branch_details.NUMERIC.apply(lambda x: safe_num(x[0]) if len(x) > 0 else None)
age = branch_details.NUMERIC.apply(lambda x: safe_age(x))
first_num = branch_details.NUMERIC.apply(lambda x: safe_num(x[0]) if len(x) > 0 else None)
second_num = branch_details.NUMERIC.apply(lambda x: safe_num(x[1]) if len(x) > 1 else None)

gt = (branch_details.GT.apply(len)>0)
gte = (branch_details.GTE.apply(len)>0)
lt = (branch_details.LT.apply(len)>0)
lte = (branch_details.LTE.apply(len)>0)

r = (branch_details.RANGE.apply(len)>0)

branch_details.loc[branch_details.has_AGE & gt, 'MIN_NUM'] = age.apply(lambda x: x + 1 if x else None)
branch_details.loc[branch_details.has_AGE & gte, 'MIN_NUM'] = age
branch_details.loc[branch_details.has_AGE & lt, 'MAX_NUM'] = age.apply(lambda x: x - 1 if x else None)
branch_details.loc[branch_details.has_AGE & lte, 'MAX_NUM'] = age

branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & gt, 'MIN_NUM'] = num.apply(lambda x: x + 1 if x else None)
branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & gte, 'MIN_NUM'] = num
branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & lt, 'MAX_NUM'] = num.apply(lambda x: x - 1 if x else None)
branch_details.loc[(branch_details.has_SIZE | branch_details.has_LAB) & lte, 'MAX_NUM'] = num

branch_details.loc[(branch_details.has_AGE | branch_details.has_SIZE | branch_details.has_LAB) & r, 'MIN_NUM'] = first_num
branch_details.loc[(branch_details.has_AGE | branch_details.has_SIZE | branch_details.has_LAB) & r, 'MAX_NUM'] = second_num

In [106]:
# very likely to be able to improve these value extraction steps through either medspacy NER or better rules / more fullsome parsing, but a decent first pass

branch_details.loc[branch_details.RULE_VALUE.isna(), 'RULE_VALUE'] = branch_details.apply(lambda row: ' '.join([t.text for t in (row.FACT_MODIFIER + row.FACT)]), axis=1)

In [107]:
branches = branch_details.apply(make_branch, axis=1)

In [108]:
with so.Session(engine) as sess:
    sess.add_all(branches)
    sess.commit()

In [109]:
variant_study = variant.merge(pointer[['regimen', 'regimen_cui']].drop_duplicates(), how='left')[['regimen', 'study', 'variant_cui', 'variant', 'regimen_cui']].drop_duplicates()
variant_study['variant_object'] = variant_study.apply(make_variant, axis=1)

In [110]:
study_object_lookup = {s.study: s.study_object for s in study.itertuples()}

In [111]:
# new issue - some null studies in variant study map
variant[variant.study.isna()]

for row in variant_study[~variant_study.study.isna()].itertuples():
    for study_name in row.study.split('|'):
        try:
            row.variant_object.studied_in.append(study_object_lookup[study_name])
        except:
            print(f'missing study: {study_name} ({row.regimen})')

missing study: No supporting study (Afatinib monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Atezolizumab monotherapy)
missing study: No supporting study (Axitinib and Avelumab)
missing study: CABL001J12301 (Bosutinib monotherapy)
missing study: No supporting study (Cabozantinib and Nivolumab)
missing study: No supporting study (Cabozantinib monotherapy)
missing study: No supporting study (Crizotinib monotherapy)
missing study: No supporting study (Crizotinib monotherapy)
missing study: CABL001J12301 (Dasatinib monotherapy)
missing study: CABL001J12301 (Imatinib monotherapy)
missing study: No supporting study (Ipilimumab and Nivolumab)
missing study: CABL001J12301 (Nilotinib monotherapy)
missing study: No supporting study (Nivolumab monotherapy)
missing study: No supporting study (Nivolumab monotherapy

In [112]:
regimen_part_sig = sig_vars[['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].drop_duplicates()

In [114]:
# spacy matcher parsing of cycle sigs

cycle_sigs_parsed = regimen_part_sig[['cyclesigs', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].fillna('').drop_duplicates().copy()
cycle_sigs_parsed['doc'] = cycle_sigs_parsed.cyclesigs.map(nlp)

for label, config in rules.items():
    cycle_sigs_parsed.doc.apply(lambda d: match_entities(matchers[label], d, label, config["MERGE"]))
    cycle_sigs_parsed[label] = cycle_sigs_parsed.doc.apply(lambda d: [t for t in d if getattr(t._, label)])
    cycle_sigs_parsed[f'has_{label}'] = (cycle_sigs_parsed[label].apply(len)>0)

cycle_sigs_parsed['full_stopping'] = cycle_sigs_parsed.apply(lambda row: get_full_stopping_condition(row.doc, row.STOPPING_CONDITION), axis=1)


cycle_sigs_parsed['DUR_MIN'] = cycle_sigs_parsed.DURATION.apply(get_min)
cycle_sigs_parsed['DUR_MAX'] = cycle_sigs_parsed.DURATION.apply(get_max)
cycle_sigs_parsed['DUR_UNITS'] = cycle_sigs_parsed.DURATION.apply(get_units)
cycle_sigs_parsed['FREQ_MIN'] = cycle_sigs_parsed.FREQUENCY.apply(get_min)
cycle_sigs_parsed['FREQ_MAX'] = cycle_sigs_parsed.FREQUENCY.apply(get_max)
cycle_sigs_parsed['FREQ_UNITS'] = cycle_sigs_parsed.FREQUENCY.apply(get_units)
cycle_sigs_parsed['REP_MIN'] = cycle_sigs_parsed.REPEATS.apply(get_min)
cycle_sigs_parsed['REP_MAX'] = cycle_sigs_parsed.REPEATS.apply(get_max)
cycle_sigs_parsed['REP_UNITS'] = cycle_sigs_parsed.REPEATS.apply(get_units)

In [115]:
reg_parts = sig_vars[['regimen', 'variant_cui','variant','phase', 'portion', 'cyclesigs', 'timing', 'timing_unit']].drop_duplicates().copy()
reg_parts['regimen_part_id'] = reg_parts.sort_values(['variant_cui', 'phase', 'portion', 'cyclesigs', 'timing', 'timing_unit']).groupby('variant_cui').cumcount()

In [116]:
reg_parts[reg_parts.cyclesigs.str.contains('2 and', na=False, case=False)]

,regimen,variant_cui,variant,phase,portion,cyclesigs,timing,timing_unit,regimen_part_id
7124,Dasatinib and Blinatumomab,131515.0,Variant #01,Consolidation,-,42-day cycle for at least 2 and 5 cycles,NaN,Cycle,0


In [117]:
regimen_parts = reg_parts.apply(make_reg_part, axis=1)
part_phase = reg_parts.apply(assign_part_phase, axis=1)
cycle_sigs_parsed['cs_residual'] = cycle_sigs_parsed.doc.apply(lambda d: [t for t in d if not t._.FREQUENCY and not t._.REPEATS and not t._.DURATION and not t._.GTE and t.text != 'for'])

In [118]:
cycle_sigs_parsed[cycle_sigs_parsed.cs_residual.apply(len) > 0]

,cyclesigs,cycle_length_lb,cycle_length_ub,cycle_length_unit,doc,AGE,has_AGE,STAGE,has_STAGE,RANGE,...,DUR_MIN,DUR_MAX,DUR_UNITS,FREQ_MIN,FREQ_MAX,FREQ_UNITS,REP_MIN,REP_MAX,REP_UNITS,cs_residual
193,Monthly cycles,1,1,month,"(Monthly, cycles)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Monthly, cycles]"
619,Continued indefinitely,1,NUB,indeterminate,"(Continued, indefinitely)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, indefinitely]"
622,Continued indefinitely unless radiotherapy pla...,,,,"(Continued, indefinitely, unless, radiotherapy...",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, indefinitely, unless, radiotherapy..."
692,Continued until achievement of CR or best resp...,1,1,week,"(Continued, until, achievement, of, CR, or, be...",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, until, achievement, of, CR, or, be..."
832,Continued until CR or 60 days,c,60,day,"(Continued, until, CR, or, 60, days)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Continued, until, CR, or, 60, days]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17563,Duration of treatment not specified,1,1,indeterminate,"(Duration, of, treatment, not, specified)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[Duration, of, treatment, not, specified]"
17599,14-day lead-in,14,14,day,"(14, -, day, lead, -, in)",[],False,[],False,[],...,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None,"[14, -, day, lead, -, in]"
17917,21-day cycles until progression or two cycles ...,21,21,day,"(21, -, day, cycles, until, progression, or, t...",[],False,[],False,[],...,NaN,NaN,None,21.0,21.0,day,2.0,2.0,cycle,"[until, progression, or, past, documented, CR]"
17949,14-day cycle for 2 years or until disease prog...,14,14,day,"(14, -, day, cycle, for, 2, years, or, until, ...",[],False,[],False,[],...,NaN,NaN,None,14.0,14.0,day,NaN,NaN,None,"[2, years, or, until, disease, progression, or..."


In [119]:
cs = cycle_sigs_parsed.apply(make_cycle_sig, axis=1)

In [120]:
# new issue - variant_cui not unique in this file any more - need to resolve the study field to unique pairs?
variant_study[variant_study.variant_cui.duplicated(keep=False)]

,regimen,study,variant_cui,variant,regimen_cui,variant_object
56,7+3d,Arlin et al. 1990|AZA-AML-001|CALGB 7421|CALGB...,129508,Variant #05,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
59,7+3d,Jin et al. 2013|Arlin et al. 1990|AZA-AML-001|...,129508,Variant #05,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
71,7+3d,AML-AZA|AML2003|AML2006|AZA-AML-001|BRIGHT AML...,129512,Variant #09,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
74,7+3d,AML-AZA|AML2003|AML2006|AZA-AML-001|CALGB 1020...,129512,Variant #09,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
75,7+3d,ECOG E1900|JHOC-J1101|SWOG S1203,129513,Variant #10,814,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
...,...,...,...,...,...,...
17824,Dara-Pd (SC daratumumab),CARTITUDE-4,144875,Variant #06,9752,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17825,Carboplatin and Paclitaxel (CP),AtTEnd|NRG/GOG-0213,144876,Variant #82,2554,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17827,Docetaxel and Trastuzumab (TH),NCIC-CTG MA.31,144876,Variant #17,6130,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17828,Fluorouracil and RT,RTOG 9704,144877,Variant #14,11359,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...


In [121]:
# RESOLVED
# # new issue - handling null condition codes
# variant_study[variant_study.study.notna() & ~variant_study.study.isin(study[study.concept_code.isna()].study.unique())].drop_duplicates(subset='variant_cui')

,regimen,study,variant_cui,variant,regimen_cui,variant_object
0,(90)YFC,MDACC ID01-233,129495,Variant #01,3071,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
3,"(90)YFC, then allo HSCT",MDACC ID01-233,129496,Variant #01,19726,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
8,131Iodine-Tositumomab monotherapy,CP-97-012|Kaminski et al. 1993|Kaminski et al....,129497,Variant #01,19847,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
13,5+2d,CALGB 7421|JHOC-J1101,129498,Variant #01,1002,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
16,5+2d,CALGB 7421,129499,Variant #02,1002,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
...,...,...,...,...,...,...
17922,Dara-Pd (SC daratumumab),CARTITUDE-4,144942,Variant #04,9752,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17923,Dara-Pd (SC daratumumab),CARTITUDE-4,144943,Variant #05,9752,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17924,Dara-Pd (SC daratumumab),CARTITUDE-4,144944,Variant #06,9752,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...
17925,Docetaxel and Trastuzumab (TH),NCIC-CTG MA.31,144945,Variant #17,6130,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...


In [123]:
from collections import defaultdict
vs_lookup = defaultdict(list)

for variant_map in variant_study[['variant_cui', 'study']].fillna('').itertuples():
    vs_lookup[variant_map.variant_cui] += variant_map.study.split('|')

In [124]:
# new issue - variant_cui - study id is not unique in this file any more - need to resolve the study field to unique pairs?

dupe_vs_map = [k for k, v in vs_lookup.items() if len(v) != len(set(v))]

In [126]:
variant_study[variant_study.variant_cui == 129495]#.variant_object.iloc[0].__dict__

,regimen,study,variant_cui,variant,regimen_cui,variant_object
0,(90)YFC,MDACC ID01-233,129495,Variant #01,3071,<hemonc_alchemy.model.hemonc_model.Hemonc_Vari...


In [127]:
with so.Session(engine) as sess:
    sess.add_all(list(variant_study[variant_study.study.notna() & 
                                    ~variant_study.study.isin(study[study.concept_code.isna()].study.unique()) & 
                                    ~variant_study.variant_cui.isin(dupe_vs_map)].drop_duplicates(subset='variant_cui').variant_object))
    sess.add_all(regimen_parts)
    sess.add_all(cs)
    sess.commit()

/var/folders/q5/f31pwsdj1ys3tz95nb2zdd540000gn/T/ipykernel_34144/182496178.py:7: SAWarning: Object of type <Hemonc_Variant> not in session, add operation along 'Hemonc_Study.studies' won't proceed
  sess.commit()


In [128]:
sig_vars[sig_vars.variant_cui==131430]#sig_vars[['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].duplicated()]

regimen_part_sig[regimen_part_sig.regimen=='D-FEC plus Bev']

reg_parts[reg_parts.variant_cui==131430]#sig_vars[['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'cycle_length_lb', 'cycle_length_ub', 'cycle_length_unit']].duplicated()]


,regimen,variant_cui,variant,phase,portion,cyclesigs,timing,timing_unit,regimen_part_id
6733,D-FEC+Bev,131430.0,Variant #01,NaN,D portion,21-day cycle for 3 cycles,Cycles 1 to 3,Cycle,0
6735,D-FEC+Bev,131430.0,Variant #01,NaN,FEC portion,21-day course,Cycle 4,Course,1
6736,D-FEC+Bev,131430.0,Variant #01,NaN,FEC portion,21-day cycle for 3 cycles,Cycles 4 to 6,Cycle,2


In [144]:
sig_vars_components = sig_vars.merge(component_df[['concept_name', 'concept_code']], left_on='component', right_on='concept_name', how='left').drop_duplicates().merge(reg_parts[['variant_cui', 'phase', 'portion', 'cyclesigs', 'regimen_part_id']])
sig_vars_components[['variant_cui', 'regimen_part_id', 'component', 'component_role', 'timing', 'step_number', 'inParens', 'class', 'tail']][sig_vars_components[['variant_cui', 'regimen_part_id', 'component', 'component_role', 'timing', 'step_number', 'class', 'tail']].duplicated()]

,variant_cui,regimen_part_id,component,component_role,timing,step_number,inParens,class,tail
6946,136544.0,0,Cyclosporine,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,adjusted to maintain therapeutic cyclosporine ...
7041,131390.0,0,Cytarabine,primary systemic,NaN,1 of 1,NaN,IV intermittent canonical Sig,-
12534,136926.0,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12535,136926.0,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12536,136926.0,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12537,136926.0,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
12538,136926.0,0,Hydroxyurea,primary systemic,NaN,1 of 1,NaN,Non-canonical Sig,-
14134,132911.0,0,Lenalidomide,primary systemic,NaN,1 of 1,NaN,Non-IV canonical Sig,-
14135,132911.0,0,Lenalidomide,primary systemic,NaN,1 of 1,NaN,Non-IV canonical Sig,-
14838,144895.0,0,Methotrexate,primary systemic,NaN,1 of 1,NaN,Non-IV canonical Sig,-


In [153]:
sig_vars_components['sig_id'] = sig_vars_components.groupby(['regimen', 'variant_cui', 'regimen_part_id']).cumcount()

In [154]:
sig_vars_components = sig_vars_components.sort_values(['variant_cui', 'regimen_part_id', 'concept_name', 'step_number'])

In [155]:
# if days are empty, but this component and regimen part are identical to the previous row when sorted in this fashion, filling these day details forward

sig_vars_components.loc[sig_vars_components.allDays.isna() & (sig_vars_components.concept_name.shift(1) ==  sig_vars_components.concept_name), 'allDays'] = sig_vars_components.allDays.shift(1)

In [156]:
sig_objects = sig_vars_components.apply(make_sig, axis=1)
sd = sig_vars_components.apply(make_sig_days, axis=1)

In [157]:
sig_vars_components.component.apply(len).max()

40

In [158]:
with so.Session(engine) as sess:
    sess.add_all(sig_objects)
    for s in sd:
        sess.add_all(s)
    sess.commit()

In [122]:
sig_vars_components[['component', 'doseMinNum', 'doseMaxNum', 'doseUnit', 'doseCapNum', 'doseCapUnit', 'divided', 'durationMinNum', 'durationMaxNum', 'durationUnit', 'frequency', 'inParens', 'sequence', 'seq.rel', 'seq.rel.what']]['seq.rel.what'].value_counts()

seq.rel.what
Elotuzumab           12
Ifosfamide            6
Fludarabine           5
Leucovorin            5
Irinotecan            3
Rituximab             3
Melphalan             2
Radiation therapy     2
HIPEC                 2
Etoposide             1
Teniposide            1
chemotherapy          1
Methotrexate          1
Radiotherapy          1
Mesna                 1
Levoleucovorin        1
Cyclophosphamide      1
Goserelin             1
Name: count, dtype: int64

In [123]:
regimen[regimen.regimen_cui.duplicated()].head()

,regimen,regimen_cui
rownames,,
145769,"Busulfan and Fludarabine, then allo HSCT",1584
133438,CYVE,1105
133514,"Arsenic trioxide, then ATRA and Daunorubicin",2391
126628,FC,2913
133621,"Fludarabine, Busulfan, ATG, Ibritumomab tiuxetan",3015


In [159]:
### Issues

# Need a n:m context <-> status mapper

# single record with >1 setting - is this a variant?
context[context.setting.map(lambda x: get_enum(Setting, x)).isna()].setting.value_counts()

# list of regimens that have no associated variants

vv = variant.merge(pointer[['regimen', 'regimen_cui']].drop_duplicates(), how='outer')
vv[vv.variant.isna()].regimen.unique()

# and vice-versa
vv[vv.regimen_cui.isna()]


# some potential duplication across variant/sig rows
dup = sig_vars[sig_vars[['study',  'regimen', 'component', 'variant_cui', 'branch', 'timing', 'step_number', 'portion', 'class']].duplicated(keep=False)]

for regimen, reg_dets in dup.groupby('regimen'):
    print(regimen)
    for lab, d in reg_dets.to_dict().items():
        if len(set(d.values())) != 1:
            print('\t', lab, d)

# should these variant definitions have the same cyclesig?

distinct_sigs = sig_vars[~sig_vars.component.str.contains('surg|brachy|radiotherapy', case=False)][['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch']].drop_duplicates()
distinct_sigs[distinct_sigs[['regimen', 'variant', 'phase', 'portion', 'timing', 'branch']].duplicated(keep=False)][['regimen', 'variant']].drop_duplicates()

sig_vars[(sig_vars.regimen=='TACE, then 5-FU') & (sig_vars.variant=='Variant #01')][['regimen', 'variant', 'phase', 'portion', 'cyclesigs', 'timing', 'branch', 'component']]


# components in the variant file that aren't in the drug class - need to revisit to pull in procedures
sig_vars_components[sig_vars_components.concept_code.isna()]

BCG vaccine monotherapy
	 portion {1203: '-', 1204: '-', 1205: 'induction', 1206: 'induction', 1207: '-', 1208: '-'}
	 timing_sequence {1203: '1,8,15', 1204: '1,8,15', 1205: '1,8,15,22,29,36', 1206: '1,8,15,22,29,36', 1207: '1,8,15,22,29,36', 1208: '1,8,15,22,29,36'}
	 timing {1203: 'Days 1, 8, 15', 1204: 'Days 1, 8, 15', 1205: 'Days 1, 8, 15, 22, 29, 36', 1206: 'Days 1, 8, 15, 22, 29, 36', 1207: 'Days 1, 8, 15, 22, 29, 36', 1208: 'Days 1, 8, 15, 22, 29, 36'}
	 cyclesigs {1203: '6-week course, then proceed to maintenance therapy', 1204: '6-week course, then proceed to maintenance therapy', 1205: '6-week course, then proceed to maintenance therapy', 1206: '6-week course, then proceed to maintenance therapy', 1207: '6-week course', 1208: '6-week course'}
	 doseMinNum {1203: '0.5', 1204: '50', 1205: '0.5', 1206: '50', 1207: '0.5', 1208: '50'}
	 doseMaxNum {1203: '0.5', 1204: '50', 1205: '0.5', 1206: '50', 1207: '0.5', 1208: '50'}
	 doseUnit_cui {1203: nan, 1204: nan, 1205: nan, 1206: nan,

,study,regimen,regimen_cui,phase,portion,component,component_cui,component_role,cycle_length_lb,cycle_length_ub,...,seq.rel.what,tail,variant,variant_cui,temp,date_added,concept_name,concept_code,regimen_part_id,sig_id
249,monarchE,Abemaciclib and ET,31832,Adjuvant,-,Antiestrogen,918,primary systemic,28,28,...,NaN,-,Variant #01,129553.0,NaN,2023-09-19,NaN,NaN,1,0
1270,TRANSFORMER,BAT,32337,NaN,-,GnRH agonist,45768,primary systemic,28,28,...,NaN,-,Variant #01,129836.0,NaN,2023-09-19,NaN,NaN,0,1
2053,Xuan et al. 2023,"Busulfan, Cyclophosphamide, Decitabine, G-CSF",119981,NaN,-,Granulocyte colony-stimulating factor,45734,primary systemic,1,1,...,NaN,-,Variant #01,130094.0,NaN,2023-09-19,NaN,NaN,0,3
6313,CALGB 40603,CP-ddAC,121944,NaN,ddAC portion,Granulocyte colony-stimulating factor,45734,secondary systemic,21,21,...,NaN,-,Variant #01,131237.0,NaN,2024-10-14,NaN,NaN,1,2
6957,R-GVHD,"Cyclosporine, Corticosteroids, Rituximab",21048,NaN,-,Steroid,45523,primary systemic,8,8,...,NaN,"of <a href=""Prednisone_.html"" title=""Prednison...",Variant #01,131374.0,NaN,2023-09-19,NaN,NaN,0,2
8469,Dreyling et al. 2004,DexaBEAM and G-CSF,24488,NaN,-,Granulocyte colony-stimulating factor,45734,primary systemic,1,1,...,NaN,-,Variant #01,131613.0,NaN,2023-09-19,NaN,NaN,0,5
9571,Hokusai VTE Cancer|Hokusai-VTE,Edoxaban monotherapy,38307,NaN,-,Low molecular weight heparin,45610,primary systemic,3,12,...,NaN,-,Variant #01,131874.0,NaN,2023-09-19,NaN,NaN,0,1
9724,HE 10/00,EP-ddCMF,120170,NaN,ddCMF portion,Granulocyte colony-stimulating factor,45734,secondary systemic,14,14,...,NaN,-,Variant #01,131916.0,NaN,2023-11-17,NaN,NaN,1,3
11063,INO-VATE ALL,FLAG,1771,NaN,-,Granulocyte colony-stimulating factor,45734,primary systemic,28,28,...,NaN,or at the institutional standard dose once per...,Variant #02,132091.0,NaN,2023-09-19,NaN,NaN,0,2
13026,CML-Study IV,Imatinib and Interferon alfa,13162,NaN,-,Interferon alfa,76030,primary systemic,7,7,...,NaN,-,Variant #01,132620.0,NaN,2023-09-19,NaN,NaN,0,0


In [160]:

distinct_sigs = sig_vars[~sig_vars.component.str.contains('surg|brachy|radiotherapy', case=False)][['regimen', 'variant', 'portion', 'cyclesigs', 'timing', 'branch']].drop_duplicates()
distinct_sigs[distinct_sigs[['regimen', 'variant', 'portion', 'timing', 'branch']].duplicated(keep=False)][['regimen', 'variant']].drop_duplicates()


,regimen,variant
632,ADT and Darolutamide,Variant #01
651,"ADT, Enzalutamide, Talazoparib",Variant #01
2434,CapeOx and Pembrolizumab,Variant #01
3661,Chlorambucil and Prednisolone,Variant #01
3665,Chlorambucil and Prednisolone,Variant #02
5609,"Cobimetinib, Vemurafenib, Atezolizumab",Variant #01
8874,ECX,Variant #04
9765,FEC,Variant #34
10992,Fotemustine monotherapy,Variant #02
12311,Ipilimumab and Sargramostim,Variant #01


In [161]:
ref[ref.concept_code.isna()].condition.value_counts()

condition
Allogeneic HSCT                           74
Autologous HSCT                           70
Stem cell mobilization regimens           16
Cellular therapy conditioning regimens     7
Name: count, dtype: int64